In [7]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 

import cartopy.crs as ccrs
import cartopy.feature as cfeature

import folium
from folium.plugins import FastMarkerCluster
from folium.plugins import HeatMap

import utm 


In [21]:
%run functions.ipynb

# 1 Data Preparation

In [23]:
# Open all files / computing intensive
base_folder_path = 'C:\\Users\\lukas\\OneDrive - Imperial College London\\0_Data\\nyc_bike_rental\\data'
start_year = 2015
end_year = 2024
combined_data = combine_csv_files_in_years(base_folder_path,start_year,end_year)
combined_data.iloc[:3]

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,...,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,61.0,2015-09-21 14:53:16,2015-09-21 14:54:17,3185.0,City Hall,40.717732,-74.043845,3185.0,City Hall,40.717732,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,290.0,2015-09-21 14:55:59,2015-09-21 15:00:50,3183.0,Exchange Place,40.716247,-74.033459,3187.0,Warren St,40.721124,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,786.0,2015-09-21 14:56:14,2015-09-21 15:09:21,3183.0,Exchange Place,40.716247,-74.033459,3183.0,Exchange Place,40.716247,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
combined_data.describe()

,Trip Duration,Start Station ID,Start Station Latitude,Start Station Longitude,End Station ID,End Station Latitude,End Station Longitude,Bike ID,Birth Year,Gender,...,end station id,end station latitude,end station longitude,bikeid,birth year,gender,start_lat,start_lng,end_lat,end_lng
count,3.396200e+05,339620.000000,339620.000000,339620.000000,339620.000000,339620.000000,339620.000000,339620.000000,313288.000000,339620.000000,...,1.329984e+06,1.329984e+06,1.329984e+06,1.329984e+06,1.312074e+06,1.329984e+06,2.479054e+06,2.479054e+06,2.474097e+06,2.474097e+06
mean,9.629629e+02,3206.818214,40.723126,-74.046422,3203.450032,40.722469,-74.045612,24911.866754,1979.186748,1.122720,...,3.272430e+03,4.069809e+01,-7.400202e+01,3.198455e+04,1.980866e+03,1.137917e+00,4.073170e+01,-7.404052e+01,4.073164e+01,-7.404025e+01
std,4.868570e+04,26.610566,0.008139,0.011194,62.608460,0.070325,0.127556,743.745408,9.652899,0.519344,...,1.656609e+02,9.896964e-01,1.799568e+00,6.323833e+03,1.048134e+01,5.445743e-01,1.216913e-02,1.213060e-02,1.229517e-02,1.212820e-02
min,6.100000e+01,3183.000000,40.692640,-74.096937,147.000000,0.000000,-74.096937,14552.000000,1900.000000,0.000000,...,7.900000e+01,0.000000e+00,-7.409694e+01,1.452900e+04,1.887000e+03,0.000000e+00,4.067833e+01,-7.408896e+01,4.064000e+01,-7.419000e+01
25%,2.470000e+02,3186.000000,40.717732,-74.050656,3186.000000,40.716540,-74.050444,24486.000000,1974.000000,1.000000,...,3.191000e+03,4.071773e+01,-7.405039e+01,2.919800e+04,1.973000e+03,1.000000e+00,4.071980e+01,-7.404629e+01,4.071959e+01,-7.404595e+01
50%,3.840000e+02,3202.000000,40.721525,-74.044247,3199.000000,40.721124,-74.043845,24602.000000,1981.000000,1.000000,...,3.205000e+03,4.071959e+01,-7.404312e+01,2.953200e+04,1.983000e+03,1.000000e+00,4.073117e+01,-7.403798e+01,4.073101e+01,-7.403798e+01
75%,6.560000e+02,3211.000000,40.727596,-74.038051,3211.000000,40.727224,-74.036486,24711.000000,1986.000000,1.000000,...,3.273000e+03,4.072722e+01,-7.403805e+01,3.361200e+04,1.989000e+03,1.000000e+00,4.074097e+01,-7.403160e+01,4.074097e+01,-7.403160e+01
max,2.026021e+07,3426.000000,40.752559,-74.032108,3442.000000,40.801343,0.000000,29296.000000,2000.000000,2.000000,...,4.071000e+03,4.084828e+01,0.000000e+00,4.998500e+04,2.004000e+03,2.000000e+00,4.086394e+01,-7.394117e+01,4.087241e+01,-7.388827e+01


In [19]:
# Open only 2024
folder_path = 'C:\\Users\\lukas\\OneDrive - Imperial College London\\0_Data\\nyc_bike_rental\\data\\2024'
data = combine_csv_files(folder_path)

In [20]:
data

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,0744109F13385D1D,electric_bike,2024-01-15 15:18:07,2024-01-15 15:32:44,Morris Canal,JC072,Oakland Ave,JC022,40.712297,-74.038185,40.737604,-74.052478,member
1,B1488BFEF9118000,classic_bike,2024-01-13 15:32:50,2024-01-13 15:36:18,JC Medical Center,JC110,Grove St PATH,JC115,40.715391,-74.049692,40.719410,-74.043090,member
2,95A2FE8E51B4C836,classic_bike,2024-01-19 13:11:00,2024-01-19 13:14:44,Morris Canal,JC072,Exchange Pl,JC116,40.712419,-74.038526,40.716366,-74.034344,member
3,95D9AFF6A1652DC1,classic_bike,2024-01-23 07:03:49,2024-01-23 07:07:11,Morris Canal,JC072,Exchange Pl,JC116,40.712419,-74.038526,40.716366,-74.034344,member
4,5F7408988A83B1B3,classic_bike,2024-01-01 16:46:10,2024-01-01 16:50:31,Morris Canal,JC072,Harborside,JC104,40.712419,-74.038526,40.719252,-74.034234,member
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50656,75E5CF948A4ADB77,electric_bike,2024-01-25 20:51:27,2024-01-25 20:59:16,Willow Ave & 12 St,HB505,Clinton St & 7 St,HB303,40.751920,-74.030396,40.745420,-74.033320,casual
50657,58DD5BF65F9B9E31,electric_bike,2024-01-27 09:03:41,2024-01-27 09:08:47,Southwest Park - Jackson St & Observer Hwy,HB401,Clinton St & 7 St,HB303,40.737551,-74.041664,40.745420,-74.033320,casual
50658,F0FBCE423BE48B08,classic_bike,2024-01-28 14:46:29,2024-01-28 14:51:43,Southwest Park - Jackson St & Observer Hwy,HB401,Clinton St & 7 St,HB303,40.737551,-74.041664,40.745420,-74.033320,member
50659,7DD51CAD2DBE7990,classic_bike,2024-01-10 08:03:47,2024-01-10 08:10:55,Southwest Park - Jackson St & Observer Hwy,HB401,Clinton St & 7 St,HB303,40.737551,-74.041664,40.745420,-74.033320,casual
